In [1]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
train_path =r'Handbags Classification/Train Dataset'
val_path =r'Handbags Classification/Valid Dataset'

In [3]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size=32, 
    class_mode='categorical',
    target_size=(150,150)
)

val_datagen = ImageDataGenerator(rescale=1/255)
val_generator = val_datagen.flow_from_directory(
    val_path,
    batch_size=32, 
    class_mode='categorical',
    target_size=(150,150)
)

Found 2749 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [4]:
image, label = train_generator.next()

In [5]:
labels = ['Backpack Min Bag', 'Belt Bag', 'Duffle Bag', 'Shoulder Bag',
       'Tote Bag']

In [6]:
data = 3
plt.imshow(image[data])
plt.title(labels[tf.argmax(label[data])])
plt.show()

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(5, activation='softmax')

])

In [8]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(512, activation = 'relu', input_dim = 784), 
#     tf.keras.layers.Dense(256, activation = 'relu'),
#     tf.keras.layers.Dense(124, activation = 'relu'),
#     tf.keras.layers.Dense(10, activation = 'softmax')
# ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 64)                5

In [9]:
model.compile(
    optimizer= 'RMSprop',
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
## callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('loss') < 1e-4): #akan berhenti apabila memenuhi
            self.model.stop_training = True
            
cb = myCallback()

In [11]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data = val_generator, callbacks = cb
)

Epoch 1/10
86/86 [==============================] - 63s 717ms/step - loss: 2.4644 - accuracy: 0.3609 - val_loss: 1.4079 - val_accuracy: 0.4120
Epoch 2/10
86/86 [==============================] - 52s 607ms/step - loss: 1.0650 - accuracy: 0.6093 - val_loss: 1.6311 - val_accuracy: 0.4020
Epoch 3/10
86/86 [==============================] - 52s 608ms/step - loss: 0.6461 - accuracy: 0.7806 - val_loss: 1.6308 - val_accuracy: 0.5300
Epoch 4/10
86/86 [==============================] - 53s 618ms/step - loss: 0.3611 - accuracy: 0.8887 - val_loss: 2.2172 - val_accuracy: 0.5360
Epoch 5/10
86/86 [==============================] - 53s 622ms/step - loss: 0.2592 - accuracy: 0.9400 - val_loss: 2.3080 - val_accuracy: 0.5540
Epoch 6/10
86/86 [==============================] - 51s 597ms/step - loss: 0.2573 - accuracy: 0.9527 - val_loss: 2.4562 - val_accuracy: 0.5600
Epoch 7/10
86/86 [==============================] - 54s 627ms/step - loss: 0.1326 - accuracy: 0.9694 - val_loss: 2.5985 - val_accuracy: 0.5660

In [12]:
#train
y_acctrain = history.history['accuracy']
y_accval = history.history['val_accuracy']

#validation
y_losstrain = history.history['loss']
y_lossval = history.history['val_loss']

#epoch
x_ = range(len(y_acctrain))

In [13]:
plt.figure(1, figsize=(5,4))
plt.plot(x_, y_acctrain, c='r', label='Training')
plt.plot(x_, y_accval, c='b', label='Validation')
plt.title('Akurasi')
plt.xlabel('Epochs')
plt.ylabel('Akurasi')
plt.legend()

plt.figure(2, figsize=(5,4))
plt.plot(x_[3:], y_losstrain[3:], c='r', label='Training')
plt.plot(x_[3:], y_lossval[3:], c='b', label='Validation')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [13]:
model.evaluate(val_generator)

16/16 [==============================] - 4s 212ms/step - loss: 3.0836 - accuracy: 0.5400


[3.083564043045044, 0.5400000214576721]

In [14]:
model.evaluate(train_generator)

86/86 [==============================] - 19s 217ms/step - loss: 0.0432 - accuracy: 0.9953


[0.04321072995662689, 0.9952710270881653]

In [15]:
result = model.predict(val_generator)
print(result)

16/16 [==============================] - 4s 200ms/step
[[1.7855466e-04 2.0968325e-02 9.7884881e-01 3.0461849e-06 1.2944224e-06]
 [3.4864068e-02 1.0306550e-16 9.6513587e-01 7.4546769e-09 1.2288917e-13]
 [9.3927848e-01 3.8332790e-05 1.1306417e-02 4.9376652e-02 4.2878615e-10]
 ...
 [3.0235201e-02 5.0028991e-03 9.5598501e-01 7.6439381e-03 1.1329246e-03]
 [9.7437727e-01 2.1367924e-02 1.6948123e-03 2.4445723e-03 1.1529719e-04]
 [3.5966149e-01 7.6928496e-04 7.4276741e-06 2.9746002e-02 6.0981584e-01]]


In [16]:
result1 = model.predict(train_generator)
print(result1)

86/86 [==============================] - 19s 216ms/step
[[1.0000000e+00 4.4947678e-12 7.5130822e-18 1.0295692e-15 1.1747867e-16]
 [2.6958093e-02 1.9673849e-02 1.6563116e-02 9.2414640e-02 8.4439027e-01]
 [3.6057073e-16 1.0476113e-04 9.9989522e-01 3.5966281e-08 1.4247095e-11]
 ...
 [2.5523197e-02 1.6731946e-02 2.5724769e-02 4.1845825e-01 5.1356190e-01]
 [1.4858678e-13 1.0000000e+00 5.9807065e-10 4.8380144e-10 8.5456847e-10]
 [3.3722185e-07 1.0934157e-03 2.4247668e-10 9.9890590e-01 3.0079380e-07]]


In [17]:
model.save('model1.h5')